In [ ]:
# import packages and key environment variables
from dotenv import load_dotenv
from chatlas import ChatOpenAI, ChatAnthropic, ChatOllama
import os

load_dotenv()
INSTANCE_ID = os.getenv('INSTANCE_ID')
API_KEY = os.getenv('API_KEY')
BASE_URL_STEM = os.getenv('BASE_URL_STEM')

In [ ]:
# set up an openai chat client
PROVIDER = "openai"
BASE_URL = f"https://{PROVIDER}.{BASE_URL_STEM}/{INSTANCE_ID}/v1"

chat = ChatOpenAI(
    model = "gpt-4o",
    api_key = API_KEY,
    base_url = BASE_URL,
    system_prompt = "You are a friendly but terse assistant.",
)

In [ ]:
# start a chat app
chat.app()

In [ ]:
# chat in the console
chat.console()

In [ ]:
# programmatic chat
chat.chat("What exactly is a spirit vegetable?")

In [ ]:
# tool (function) calling - function to get current temperature
import requests

# function to get the current temperature in a place
def get_current_temperature(place: str) -> str:
  """Get the current temperature in a given place."""
  base_url = f"https://wttr.in/{place}?format=j1"
  response = requests.get(base_url)
  data = response.json()
  return f"The current temperature in {place} is {data['current_condition'][0]['temp_C']} degrees Celsius"

# test the function
get_current_temperature("London")

In [ ]:
# register the function with your chat
chat.register_tool(get_current_temperature)

In [ ]:
# now the chat can use it
chat.chat("I'm in Atlanta today and I'm told I should wear warm clothes.  What do you think?")

In [ ]:
# pulling structured data out of text
from pydantic import BaseModel, Field

class Person(BaseModel):
    name: str
    pets: int
    skills: list[str]

chat.extract_data(
  "My name is Keith.  I have two cats and one dog named Bertie.  I am very good at Math and Computer Games", 
  data_model=Person,
)

In [ ]:
# now try a Anthropic chat client
PROVIDER = "anthropic"
BASE_URL = f"https://{PROVIDER}.{BASE_URL_STEM}/{INSTANCE_ID}"

chat = ChatAnthropic(
    api_key = API_KEY,
    system_prompt = "You are a friendly but terse assistant.",
    kwargs = {"base_url": BASE_URL}  
)

In [ ]:
chat.chat("Which integer is commonly quoted as the answer to the meaning of life?")

In [ ]:
chat.register_tool(get_current_temperature)

In [ ]:
chat.chat("My sister is heading to the capital of Norway next week?  How should she pack?")

In [ ]:
# structured data
class Person(BaseModel):
    name: str
    pets: int
    areas_of_expertise: list[str]
    qualifications: list[str]

chat.extract_data(
  """
  My name is Keith.  I had two cats and one dog named Bertie, but I recently sold the two cats.  I have a PhD in Pure Mathematics, and I am also a holder of the Licentiate Performing Diploma from Trinity College of Music in London, taking my final exams in the Recorder.
  """, 
  data_model = Person,
)

In [ ]:
# add more information
chat.extract_data(
  """
  Oh it's Keith again.  I'm sorry, I'm so dumb.  I forgot to mention I also have a new puppy which we only just picked up this week.
  """, 
  data_model = Person,
)

In [ ]:
# add more information
chat.extract_data(
  """
  It's Keith again.  I just spoke to my wife and she left the door open and the puppy ran out on the street and got killed.  It's a bad day.
  """, 
  data_model = Person,
)

In [ ]:
# local ollama models
chat = ChatOllama(
    model = "llama3.1:8b",
    system_prompt = "You are a friendly but terse assistant.",
)

In [ ]:
chat.chat("Explain Buddhism in two sentences")

In [ ]:
chat.extract_data(
      """
  My name is Keith.  I had two cats and one dog named Bertie, but I recently sold the two cats.  I have a PhD in Pure Mathematics, and I am also a holder of the Licentiate Performing Diploma from Trinity College of Music in London, taking my final exams in the Recorder.
  """, 
  data_model = Person,
)

In [ ]:
chat.register_tool(get_current_temperature)

In [ ]:
chat.chat("I'm heading Tasmania tomorrow but I forgot my sunscreen.  Should I be concerned?")